In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [3]:
#load the data from the csv file
data = pd.read_csv('../Market_Crash_Predictor/cleaned_data/final_standardized_data.csv')


In [4]:
#Show the first 5 rows of the data
data.head()

,Date,Federal Funds Rate,Unemployment Rate,GDP,Inflation Rate,SP500
0,1975-01-01,3.87,8.1,1616.116,9.143147,72.564091
1,1975-02-01,6.76,8.1,1616.116,9.143147,80.096842
2,1975-03-01,6.07,8.6,1616.116,9.143147,83.831999
3,1975-04-01,5.48,8.8,1651.853,9.143147,84.723182
4,1975-05-01,5.55,9.0,1651.853,9.143147,90.096190


In [5]:
#Make sure Date is in datetime format
data['Date'] = pd.to_datetime(data['Date'])

#Set the Date as the index
data.set_index('Date', inplace=True)

In [6]:
#Create a month-to-month percentage change in the features
data['sp500_pct_change'] = data['SP500'].pct_change()
data['gdp_pct_change'] = data['GDP'].pct_change()
data['inflation_pct_change'] = data['Inflation Rate'].pct_change()
data['unemployment_pct_change'] = data['Unemployment Rate'].pct_change()
data['fed_funds_rate_pct_change'] = data['Federal Funds Rate'].pct_change()

/var/folders/4w/dqq6x1t91hvgn2hnhb1zlk9r0000gn/T/ipykernel_38856/3432497560.py:3: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  data['gdp_pct_change'] = data['GDP'].pct_change()
/var/folders/4w/dqq6x1t91hvgn2hnhb1zlk9r0000gn/T/ipykernel_38856/3432497560.py:4: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  data['inflation_pct_change'] = data['Inflation Rate'].pct_change()
/var/folders/4w/dqq6x1t91hvgn2hnhb1zlk9r0000gn/T/ipykernel_38856/3432497560.py:5: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-lea

In [7]:
#Do a moving average and volatility calulation for the SP500
data['sp500_quarter'] = data['SP500'].rolling(window=3).mean()
data['sp500_yearly'] = data['SP500'].rolling(window=12).mean()
data['sp500_volatility_quarter'] = data['sp500_pct_change'].rolling(window=3).std()
data['sp500_volatility_yearly'] = data['sp500_pct_change'].rolling(window=12).std()


In [8]:
#Set crash threshold to 10% drop in the SP500
crash_threshold = -0.10
data['crash'] = (data['sp500_pct_change'] < crash_threshold).astype(int)


In [9]:
#Show the first 5 rows of the data
data.head()

,Federal Funds Rate,Unemployment Rate,GDP,Inflation Rate,SP500,sp500_pct_change,gdp_pct_change,inflation_pct_change,unemployment_pct_change,fed_funds_rate_pct_change,sp500_quarter,sp500_yearly,sp500_volatility_quarter,sp500_volatility_yearly,crash
Date,,,,,,,,,,,,,,,
1975-01-01,3.87,8.1,1616.116,9.143147,72.564091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1975-02-01,6.76,8.1,1616.116,9.143147,80.096842,0.103808,0.000000,0.0,0.000000,0.746770,NaN,NaN,NaN,NaN,0
1975-03-01,6.07,8.6,1616.116,9.143147,83.831999,0.046633,0.000000,0.0,0.061728,-0.102071,78.830977,NaN,NaN,NaN,0
1975-04-01,5.48,8.8,1651.853,9.143147,84.723182,0.010631,0.022113,0.0,0.023256,-0.097199,82.884008,NaN,0.046988,NaN,0
1975-05-01,5.55,9.0,1651.853,9.143147,90.096190,0.063418,0.000000,0.0,0.022727,0.012774,86.217124,NaN,0.026971,NaN,0


In [10]:
#Drop the first 12 rows of the data since they have NaN values
data = data[12:]

In [11]:
#Show the first 5 rows of the data
data.head()

,Federal Funds Rate,Unemployment Rate,GDP,Inflation Rate,SP500,sp500_pct_change,gdp_pct_change,inflation_pct_change,unemployment_pct_change,fed_funds_rate_pct_change,sp500_quarter,sp500_yearly,sp500_volatility_quarter,sp500_volatility_yearly,crash
Date,,,,,,,,,,,,,,,
1976-01-01,5.37,7.9,1820.487,5.744813,96.859524,0.091985,0.033293,-0.371681,-0.036585,0.011299,91.875081,88.184867,0.054999,0.049154,0
1976-02-01,4.84,7.7,1820.487,5.744813,100.639474,0.039025,0.000000,0.000000,-0.025316,-0.098696,95.399817,89.896753,0.053570,0.042928,0
1976-03-01,5.21,7.6,1820.487,5.744813,101.084348,0.004420,0.000000,0.000000,-0.012987,0.076446,99.527782,91.334449,0.044101,0.042277,0
1976-04-01,4.83,7.7,1852.332,5.744813,101.929524,0.008361,0.017493,0.000000,0.013158,-0.072937,101.217782,92.768311,0.018944,0.042310,0
1976-05-01,5.10,7.4,1852.332,5.744813,101.161999,-0.007530,0.000000,0.000000,-0.038961,0.055901,101.391957,93.690462,0.008275,0.040030,0


In [12]:
#Define the features and the target
features = [
    'sp500_pct_change', 'gdp_pct_change', 'inflation_pct_change', 
    'unemployment_pct_change', 'fed_funds_rate_pct_change',
    'sp500_quarter', 'sp500_yearly', 'sp500_volatility_quarter', 'sp500_volatility_yearly'
]

X = data[features]
y = data['crash']

In [13]:
#Standardize the features
scaler = StandardScaler()
x_scaled = scaler.fit_transform(X)

In [14]:
#SPlit the data into training and testing sets
train_data = data[(data.index < '2008-01-01')]
test_data = data[(data.index >= '2008-01-01')]

X_train = scaler.fit_transform(train_data[features])
y_train = train_data['crash']
X_test = scaler.transform(test_data[features])
y_test = test_data['crash']

In [15]:
#Create a logistic regression model
model = LogisticRegression()

#Fit the model to the training data
model.fit(X_train, y_train)

LogisticRegression()

In [16]:
#Make predictions on the testing data
y_pred = model.predict(X_test)

#Print the classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99       200
           1       0.40      0.67      0.50         3

    accuracy                           0.98       203
   macro avg       0.70      0.83      0.74       203
weighted avg       0.99      0.98      0.98       203



In [17]:
#Show the coefficients of the model
coefficients = pd.DataFrame({
	'Variable': features,
	'Coefficient': model.coef_[0]
})

#Transform the coefficients into absolute values
coefficients['Coefficient'] = coefficients['Coefficient'].abs()

In [18]:
#Print the coefficients
print(coefficients)

                    Variable  Coefficient
0           sp500_pct_change     1.764696
1             gdp_pct_change     0.349299
2       inflation_pct_change     0.021158
3    unemployment_pct_change     0.065704
4  fed_funds_rate_pct_change     0.192824
5              sp500_quarter     0.173619
6               sp500_yearly     0.289815
7   sp500_volatility_quarter     0.377890
8    sp500_volatility_yearly     0.979962


In [19]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Calculate predicted probabilities of a market crash
y_prob = model.predict_proba(X_test)[:, 1]  # Probability of the positive class (market crash)

# Add the predicted probabilities to the test data for visualization
test_data['crash_probability'] = y_prob
test_data['Date'] = test_data.index

# Create a Plotly scatter plot to visualize the data and clusters of probabilities
fig = px.scatter(test_data, x='Date', y='sp500_pct_change', color='crash_probability',
                 title='Market Crash Probability over Time with Clusters',
                 labels={'crash_probability': 'Crash Probability', 'sp500_pct_change': 'SP500 % Change'},
                 color_continuous_scale='Viridis')

# Add markers for actual crashes
fig.add_trace(go.Scatter(x=test_data[test_data['crash'] == 1]['Date'],
                         y=test_data[test_data['crash'] == 1]['sp500_pct_change'],
                         mode='markers',
                         name='Actual Crash',
                         marker=dict(symbol='x', color='red', size=10)))

# Update layout for clarity
fig.update_layout(coloraxis_colorbar=dict(title='Crash Probability'),
                  xaxis_title='Date',
                  yaxis_title='S&P 500 % Change',
                  template='plotly_dark')

# Show the interactive plot
fig.show()


/var/folders/4w/dqq6x1t91hvgn2hnhb1zlk9r0000gn/T/ipykernel_38856/196111741.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['crash_probability'] = y_prob
/var/folders/4w/dqq6x1t91hvgn2hnhb1zlk9r0000gn/T/ipykernel_38856/196111741.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['Date'] = test_data.index
